In [1]:
import torch
from torch import optim, nn
from torch.autograd import Variable
# import pixiedust

In [2]:
x = torch.tensor([[0,0,0,0],[1,0,0,0],[0,1,0,0],[0,0,1,0],[1,1,0,0],[1,0,1,0],[0,1,1,0],[1,1,1,0],[0,0,0,1],[1,0,0,1],[0,1,0,1],[0,0,1,1],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1.]])
x.requires_grad_()
target = torch.tensor([[0],[0],[0],[0],[0],[0],[0],[0],[1],[1],[1],[1],[1],[1],[1],[1.]])


#   Variables for performance metrics
epochs = 20
lr = 0.2
counter = 0

# Define 2 chained models
models = [
    nn.Sequential(
        nn.Linear(4, 3),
        nn.Tanh()
    ),
    nn.Sequential(
        nn.Linear(3, 1),
        nn.Sigmoid()
    )
]

# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(params=model.parameters(),lr=lr)
    for model in models
]

In [3]:
def train():
    # Training Logic
    for iter in range(epochs):

        # 1) erase previous gradients (if they exist)
        for opt in optimizers:
            opt.zero_grad()

        # 2) make a prediction
        a  = models[0](x)

        # 3) send the activation signal to the next model
        remote_a = a.detach()
        # re-enable autograd here
        remote_a.requires_grad_()

        pred =  models[1](remote_a)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()
        
        # 5) Backprop gradient to model behind
        grad_a = remote_a.grad.clone()
        
        # 5) Go backward from grad_a 
        print(models[0][0].weight.grad)
        a.backward(grad_a)
        print(models[0][0].weight.grad)

        # 5) change the weights
        for opt in optimizers:
            opt.step()

        # 6) print our progress
        # Do not use .data
        print(loss.detach())
        
train()

None
tensor([[-0.0053, -0.0080, -0.0099,  0.1347],
        [ 0.0033,  0.0025,  0.0027,  0.4079],
        [-0.0140, -0.0167, -0.0184, -1.0661]])
tensor(4.5665)
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[-0.0505, -0.0569, -0.0608,  0.3359],
        [-0.0096, -0.0110, -0.0119,  0.2134],
        [ 0.0131,  0.0151,  0.0165, -0.6925]])
tensor(4.0536)
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[-0.0955, -0.1044, -0.1091,  0.4753],
        [-0.0078, -0.0090, -0.0097,  0.1344],
        [ 0.0182,  0.0216,  0.0238, -0.4894]])
tensor(3.7991)
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[-0.1204, -0.1336, -0.1392,  0.6185],
        [-0.0045, -0.0056, -0.0063,  0.1023],
        [ 0.0141,  0.0185,  0.0212, -0.3982]])
tensor(3.5617)
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[-0.1124, -0.1321, -0.1394,  0.7706],
        [-0.0013,